### Key idea: Amino acid composition

#### Thay vì so sánh từng chữ cái A-T-G-C (như BLAST), chúng ta sẽ biến protein thành các con số thống kê (Features).
- Giả thuyết: Các protein có cùng chức năng thường có tỉ lệ các axit amin giống nhau. 
- Ví dụ: Protein ưa nước sẽ có nhiều axit amin loại D, E, K, R.
- Phương pháp: Đếm tần suất xuất hiện của 20 loại axit amin trong chuỗi.
- Kết quả: Mỗi protein biến thành một vector có 20 chiều (hoặc nhiều hơn nếu thêm độ dài, khối lượng...).

### Feature extraction

In [8]:
import pandas as pd
import numpy as np
from Bio import SeqIO
from tqdm import tqdm
from collections import Counter

AMINO_ACIDS = 'ACDEFGHIKLMNPQRSTVWY'

def clean_id(raw_id):
    if "|" in str(raw_id):
        return raw_id.split("|")[1]
    return raw_id

def get_amino_acid_composition(fasta_path, limit=None):
    data = []
    ids = []
    
    print(f"Đang xử lý file: {fasta_path}")
    for i, record in enumerate(tqdm(SeqIO.parse(fasta_path, "fasta"))):
        if limit and i >= limit: break
        
        seq = str(record.seq)
        length = len(seq)
        if length == 0: continue
            
        # Đếm tần suất
        counts = Counter(seq)
        features = [counts.get(aa, 0) / length for aa in AMINO_ACIDS]
        features.append(length)
        
        data.append(features)
        
        # Clean ID ngay khi đọc ---
        ids.append(clean_id(record.id)) 
        
    cols = list(AMINO_ACIDS) + ["Length"]
    return ids, pd.DataFrame(data, columns=cols)

# --- CHẠY THỬ ---
train_fasta = "/workspace/data/Train/train_sequences.fasta"
train_ids, X_train = get_amino_acid_composition(train_fasta, limit=50000)

print("Shape X_train:", X_train.shape)
X_train.head()

Đang xử lý file: /workspace/data/Train/train_sequences.fasta


50000it [00:00, 59538.20it/s]

Shape X_train: (50000, 21)


,A,C,D,E,F,G,H,I,K,L,...,N,P,Q,R,S,T,V,W,Y,Length
0,0.000000,0.000000,0.000000,0.062500,0.062500,0.062500,0.000000,0.062500,0.062500,0.062500,...,0.000000,0.062500,0.062500,0.187500,0.000000,0.000000,0.000000,0.062500,0.125000,16
1,0.049863,0.017077,0.060109,0.066940,0.035519,0.037568,0.032787,0.057377,0.071038,0.099044,...,0.053279,0.045765,0.047814,0.034836,0.120902,0.058060,0.056011,0.008197,0.023907,1464
2,0.053356,0.037866,0.063683,0.092943,0.015491,0.056799,0.034423,0.036145,0.092943,0.104991,...,0.060241,0.027539,0.051635,0.049914,0.080895,0.037866,0.046472,0.008606,0.017212,581
3,0.107759,0.021552,0.021552,0.051724,0.064655,0.073276,0.021552,0.081897,0.043103,0.133621,...,0.025862,0.030172,0.038793,0.025862,0.056034,0.030172,0.107759,0.021552,0.025862,232
4,0.058524,0.013995,0.047074,0.090331,0.053435,0.049618,0.026718,0.045802,0.077608,0.090331,...,0.044529,0.063613,0.043257,0.055980,0.080153,0.053435,0.052163,0.008906,0.021628,786


### Labels 

In [9]:
train_terms = pd.read_csv("/workspace/data/Train/train_terms.tsv", sep="\t", usecols=["EntryID", "term"])

# Lọc chỉ lấy các protein có trong X_train (vì nãy mình limit 50k)
train_ids_set = set(train_ids)
train_terms_filtered = train_terms[train_terms["EntryID"].isin(train_ids_set)]

# Tìm Top 100 terms phổ biến nhất
top_100_terms = train_terms_filtered["term"].value_counts().head(100).index.tolist()
print(f"Đã chọn {len(top_100_terms)} nhãn phổ biến nhất để train.")

# Tạo Pivot Table (One-hot encoding cho Multilabel)
# Dòng: ProteinID, Cột: Term, Giá trị: 1 nếu có, 0 nếu không
print("Đang tạo ma trận Y (Label Matrix)...")
Y_matrix = train_terms_filtered[train_terms_filtered["term"].isin(top_100_terms)] \
            .pivot_table(index="EntryID", columns="term", aggfunc="size", fill_value=0)

# Đảm bảo thứ tự của Y khớp với X_train
# (Bước này quan trọng: Protein ở dòng 1 của X phải khớp với dòng 1 của Y)
Y_train = Y_matrix.reindex(train_ids).fillna(0).astype(int)

print("Shape Y_train:", Y_train.shape)

Đã chọn 100 nhãn phổ biến nhất để train.
Đang tạo ma trận Y (Label Matrix)...
Shape Y_train: (50000, 100)


In [10]:
Y_train

term,GO:0000122,GO:0000139,GO:0000287,GO:0000785,GO:0000976,GO:0000978,GO:0000981,GO:0001228,GO:0001666,GO:0003677,...,GO:0045202,GO:0045892,GO:0045893,GO:0045944,GO:0046982,GO:0048471,GO:0061630,GO:0070062,GO:0098978,GO:1990837
EntryID,,,,,,,,,,,,,,,,,,,,,
A0A0C5B5G6,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
A0JNW5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A0JP26,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A0PK11,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A1A4S6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
P13619,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
P13621,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
P13902,0,0,0,0,0,1,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0


### Training

In [5]:
%pip install scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 65.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 62.4 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [scikit-learn] [scikit-learn]
Note: you may need to restart the kernel to use updated packages.


In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

# Chia tập train/val để đánh giá nội bộ
X_t, X_v, Y_t, Y_v = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

print("Đang train Random Forest...")
# n_jobs=-1 để dùng hết tất cả các nhân CPU
clf = RandomForestClassifier(n_estimators=50, n_jobs=-1, verbose=1)
clf.fit(X_t, Y_t)

# Đánh giá sơ bộ
print("Đang predict tập Val...")
Y_pred_val = clf.predict(X_v)
print("Model Score (Mean Accuracy):", clf.score(X_v, Y_v))
# Lưu ý: Accuracy ở multi-label thường rất thấp (cần đúng hết 100 nhãn mới tính là đúng),
# nên đừng lo nếu thấy số thấp.

Đang train Random Forest...


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 31 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 out of  50 | elapsed:    4.9s remaining:    1.2s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    5.3s finished
[Parallel(n_jobs=31)]: Using backend ThreadingBackend with 31 concurrent workers.


Đang predict tập Val...


[Parallel(n_jobs=31)]: Done  40 out of  50 | elapsed:    0.3s remaining:    0.1s
[Parallel(n_jobs=31)]: Done  50 out of  50 | elapsed:    0.4s finished
[Parallel(n_jobs=31)]: Using backend ThreadingBackend with 31 concurrent workers.


Model Score (Mean Accuracy): 0.1204


[Parallel(n_jobs=31)]: Done  40 out of  50 | elapsed:    0.3s remaining:    0.1s
[Parallel(n_jobs=31)]: Done  50 out of  50 | elapsed:    0.4s finished


### Validation

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score

X_hoc, X_thi, Y_hoc, Y_thi = train_test_split(
    X_train, 
    Y_train, 
    test_size=0.2, 
    random_state=42

In [14]:
clf.fit(X_hoc, Y_hoc)

Y_pred = clf.predict(X_thi)

local_score = f1_score(Y_thi, Y_pred, average='micro')

print(f"Local F1: {local_score:.4f}")

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 31 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 out of  50 | elapsed:    5.1s remaining:    1.3s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    5.3s finished
[Parallel(n_jobs=31)]: Using backend ThreadingBackend with 31 concurrent workers.


Local F1: 0.2699


[Parallel(n_jobs=31)]: Done  40 out of  50 | elapsed:    0.3s remaining:    0.1s
[Parallel(n_jobs=31)]: Done  50 out of  50 | elapsed:    0.4s finished


### Inference

In [12]:
test_fasta = "/workspace/data/Test/testsuperset.fasta"
test_ids, X_test = get_amino_acid_composition(test_fasta)

# Dự đoán xác suất (Probability)
# Random Forest trả về list các array xác suất cho từng class
print("Đang dự đoán tập Test...")
# predict_proba trả về list gồm 100 array (mỗi array cho 1 term)
# Ta cần format lại
probas = clf.predict_proba(X_test) 

# Ghi file Submission
output_lines = []
terms_columns = Y_train.columns # Danh sách tên các GO term (Top 100)

print("Đang ghi file submission...")
# probas là list n_labels phần tử, mỗi phần tử là array (n_samples, 2)
# probas[i][:, 1] là xác suất của label i
# Ta cần transpose lại để duyệt theo từng protein

threshold = 0.1 # Chỉ lấy nhãn có xác suất > 10%

for i, pid in enumerate(tqdm(test_ids)):
    for j, term in enumerate(terms_columns):
        # Lấy xác suất của class "Positive" (cột index 1)
        # Một số trường hợp model chỉ trả về 1 cột (nếu class đó toàn 0 hoặc toàn 1)
        if probas[j].shape[1] == 2:
            score = probas[j][i, 1]
        else:
            score = 0.0 
            
        if score > threshold:
            output_lines.append(f"{pid}\t{term}\t{score:.3f}")

with open("submission_level3_rf.tsv", "w") as f:
    f.write("\n".join(output_lines))

Đang xử lý file: /workspace/data/Test/testsuperset.fasta


224309it [00:02, 76131.40it/s] 


Đang dự đoán tập Test...


[Parallel(n_jobs=31)]: Using backend ThreadingBackend with 31 concurrent workers.
[Parallel(n_jobs=31)]: Done  40 out of  50 | elapsed:   11.3s remaining:    2.8s
[Parallel(n_jobs=31)]: Done  50 out of  50 | elapsed:   12.8s finished


Đang ghi file submission...


100%|██████████| 224309/224309 [00:04<00:00, 47935.59it/s]
